In [104]:
# %load build_out.py
#!/usr/bin/env python3
"""
@author: craig
"""
import pandas as pd
import numpy as np
import sys
np.random.seed(23)
pd.set_option('display.max_columns', None)

con = pd.read_csv('csv/conditions.csv')
con.columns = [col.lower() for col in con.columns]

# =============================================================================
# Get Diabetic Patients
# =============================================================================
mask = con['description'].str.contains('iabet')

diabetics = list(con[mask]['patient'].unique()) # length 37884

# =============================================================================
# Get observations for diabetic patients
# =============================================================================
obs = pd.read_csv('csv/observations.csv')
obs.columns = [col.lower() for col in obs.columns]

mask_ob = obs['patient'].isin(diabetics)
obs[mask_ob]

# =============================================================================
# Find prevalent biomarkers among diabetic patients
# =============================================================================
biomarks = [k for k,v 
            in dict(obs[mask_ob]['description'].value_counts()).items()
            if v > 4000
]

mask_ob_mark = obs['description'].isin(biomarks)

dfobs = obs[mask_ob & mask_ob_mark][['date',
                                     'encounter',
                                       'patient',
                                       'description', 
                                       'value'
                                       ]
]

dfobs.index = list(range(dfobs.shape[0]))

del obs
del diabetics
del biomarks
del mask
del mask_ob
del mask_ob_mark

In [105]:
dfobs.head()

,date,encounter,patient,description,value
0,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,Hemoglobin A1c/Hemoglobin.total in Blood,5.9
1,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,Body Height,181.93
2,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,Body Weight,97.66
3,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,Body Mass Index,29.5
4,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,Systolic Blood Pressure,129.0


In [106]:
# =============================================================================
# Pivot the selected data to get redings for each patient
# =============================================================================
dfobs.reset_index(inplace=True)

dfobs_pivot = dfobs.pivot(index='index', columns='description', values='value')

diabetic_readings = pd.concat([dfobs, dfobs_pivot], axis=1)

In [107]:
# =============================================================================
# Trim the diabetic readings data frame
# =============================================================================
diabetic_readings.drop(['description', 'value'], axis=1, inplace=True)

diabetic_readings[list(diabetic_readings.columns[4:])] = \
    diabetic_readings[list(diabetic_readings.columns[4:])] \
    .applymap(lambda x: 0 if type(x)!=str else float(x))

In [108]:
diabetic_readings.head()

,index,date,encounter,patient,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Chloride,Creatinine,Diastolic Blood Pressure,Estimated Glomerular Filtration Rate,FEV1/​FVC,Glucose,Hemoglobin A1c/Hemoglobin.total in Blood,High Density Lipoprotein Cholesterol,Low Density Lipoprotein Cholesterol,Microalbumin Creatine Ratio,Oral temperature,Potassium,Sodium,Systolic Blood Pressure,Total Cholesterol,Triglycerides,Urea Nitrogen
0,0,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,181.93,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,0.00,0.0,97.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,0.00,29.5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2010-07-21,0afe4a0e-549c-413f-91c9-0fa6abdcf412,33025806-4630-46da-a5ff-ee0d9e20465a,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129.0,0.0,0.0,0.0


In [109]:
diabetic_readings.groupby(['encounter', 'patient', 'date']).sum().max()[1:] \
== diabetic_readings.max()[4:]

Body Height                                  True
Body Mass Index                              True
Body Weight                                  True
Calcium                                      True
Carbon Dioxide                               True
Chloride                                     True
Creatinine                                   True
Diastolic Blood Pressure                     True
Estimated Glomerular Filtration Rate         True
FEV1/​FVC                                    True
Glucose                                      True
Hemoglobin A1c/Hemoglobin.total in Blood    False
High Density Lipoprotein Cholesterol         True
Low Density Lipoprotein Cholesterol          True
Microalbumin Creatine Ratio                  True
Oral temperature                             True
Potassium                                    True
Sodium                                       True
Systolic Blood Pressure                      True
Total Cholesterol                            True


Note: the Hemoglobin column will need to be dropped

In [110]:
# diabetic_readings.groupby(['encounter', 'patient', 'date']).sum()
#     ['Hemoglobin A1c/Hemoglobin.total in Blood'] \
#     .count()

In [111]:
readings = diabetic_readings.groupby(['encounter', 'patient', 'date']).sum()

In [112]:
readings.head()

,,,index,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Chloride,Creatinine,Diastolic Blood Pressure,Estimated Glomerular Filtration Rate,FEV1/​FVC,Glucose,Hemoglobin A1c/Hemoglobin.total in Blood,High Density Lipoprotein Cholesterol,Low Density Lipoprotein Cholesterol,Microalbumin Creatine Ratio,Oral temperature,Potassium,Sodium,Systolic Blood Pressure,Total Cholesterol,Triglycerides,Urea Nitrogen
encounter,patient,date,,,,,,,,,,,,,,,,,,,,,,,
000002a3-8783-4147-991c-99c890603323,7b0b6f94-d066-4962-be0b-b038229b94c4,2014-07-24,34729800,174.55,32.87,100.16,8.91,29.0,104.0,1.0,74.0,0.0,0.0,71.0,12.8,0.0,0.0,0.0,0.0,3.75,141.0,114.0,0.0,0.0,18.0
0000506d-a912-4078-9041-a61935178b60,6d4c4136-a178-462b-9ce6-7f316989f1d3,2013-01-06,8796000,165.09,30.43,82.93,10.18,28.0,109.0,1.0,84.0,0.0,0.0,77.0,11.8,0.0,0.0,0.0,0.0,4.39,143.0,125.0,0.0,0.0,9.0
00008813-ef8d-4787-8929-70913483b217,154f808f-6a7f-44ed-bde3-2df3d8beb7d9,2011-11-10,12464508,174.96,32.20,98.57,9.86,29.0,106.0,1.0,72.0,60.0,0.0,114.0,10.0,59.0,111.0,157.0,0.0,4.16,144.0,122.0,206.0,180.0,19.0
00014417-effa-4386-a8ea-a2bd4a1d2e99,109cd982-c84e-42ad-b130-5a90a00cbba6,2011-10-11,38711064,174.21,28.17,85.49,9.43,22.0,106.0,1.0,119.0,60.0,0.0,84.0,12.6,0.0,0.0,0.0,0.0,4.14,136.0,141.0,0.0,0.0,15.0
00016d13-7ce2-4f74-b2e8-a772d3dce15f,93eb91e5-79aa-4443-b9b6-67834534ef41,2015-03-31,17215020,177.03,29.40,92.14,9.19,28.0,110.0,1.0,86.0,0.0,0.0,96.0,12.4,0.0,0.0,0.0,0.0,5.01,142.0,103.0,0.0,0.0,14.0


In [113]:
readings.index[0][2]

'2014-07-24'

In [114]:
# =============================================================================
# Reconstitute the grouped data frame 
# =============================================================================
encounters = [readings.index[i][0] for i in range(readings.shape[0])]

readings['encounter'] = encounters

patients = [readings.index[i][1] for i in range(readings.shape[0])]

readings['patient'] = patients

date = [readings.index[i][2] for i in range(readings.shape[0])]

readings['obs_date'] = date

readings.index = list(range(readings.shape[0]))

readings.drop(['index', 'Hemoglobin A1c/Hemoglobin.total in Blood'], axis=1, inplace=True)

In [115]:
# =============================================================================
# Coerce the artificially created zeros to nulls, there were no zeros
# in the original observations data set
# (obs['value']==0).any() # returns False
# =============================================================================
readings = readings.applymap(lambda x: np.nan if x==0 else x)

In [116]:
readings[readings['Body Height'] > 242]

,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Chloride,Creatinine,Diastolic Blood Pressure,Estimated Glomerular Filtration Rate,FEV1/​FVC,Glucose,High Density Lipoprotein Cholesterol,Low Density Lipoprotein Cholesterol,Microalbumin Creatine Ratio,Oral temperature,Potassium,Sodium,Systolic Blood Pressure,Total Cholesterol,Triglycerides,Urea Nitrogen,encounter,patient,obs_date


In [117]:
readings.head()

,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Chloride,Creatinine,Diastolic Blood Pressure,Estimated Glomerular Filtration Rate,FEV1/​FVC,Glucose,High Density Lipoprotein Cholesterol,Low Density Lipoprotein Cholesterol,Microalbumin Creatine Ratio,Oral temperature,Potassium,Sodium,Systolic Blood Pressure,Total Cholesterol,Triglycerides,Urea Nitrogen,encounter,patient,obs_date
0,174.55,32.87,100.16,8.91,29.0,104.0,1.0,74.0,NaN,NaN,71.0,NaN,NaN,NaN,NaN,3.75,141.0,114.0,NaN,NaN,18.0,000002a3-8783-4147-991c-99c890603323,7b0b6f94-d066-4962-be0b-b038229b94c4,2014-07-24
1,165.09,30.43,82.93,10.18,28.0,109.0,1.0,84.0,NaN,NaN,77.0,NaN,NaN,NaN,NaN,4.39,143.0,125.0,NaN,NaN,9.0,0000506d-a912-4078-9041-a61935178b60,6d4c4136-a178-462b-9ce6-7f316989f1d3,2013-01-06
2,174.96,32.20,98.57,9.86,29.0,106.0,1.0,72.0,60.0,NaN,114.0,59.0,111.0,157.0,NaN,4.16,144.0,122.0,206.0,180.0,19.0,00008813-ef8d-4787-8929-70913483b217,154f808f-6a7f-44ed-bde3-2df3d8beb7d9,2011-11-10
3,174.21,28.17,85.49,9.43,22.0,106.0,1.0,119.0,60.0,NaN,84.0,NaN,NaN,NaN,NaN,4.14,136.0,141.0,NaN,NaN,15.0,00014417-effa-4386-a8ea-a2bd4a1d2e99,109cd982-c84e-42ad-b130-5a90a00cbba6,2011-10-11
4,177.03,29.40,92.14,9.19,28.0,110.0,1.0,86.0,NaN,NaN,96.0,NaN,NaN,NaN,NaN,5.01,142.0,103.0,NaN,NaN,14.0,00016d13-7ce2-4f74-b2e8-a772d3dce15f,93eb91e5-79aa-4443-b9b6-67834534ef41,2015-03-31


Incorporate the Encounters, Patients, Conditions, and Procedures Data Sets.

In [118]:
enc = pd.read_csv('./csv/encounters.csv')
enc.columns = [col.lower() for col in enc.columns]
enc_map = dict([(col, 'enc_'+col) for col in enc.columns if col not in ['encounter', 'patient']])
enc_map.update({'id':'encounter'})
enc.rename(mapper=enc_map, axis='columns', inplace=True)

enc.head()

,encounter,enc_date,patient,enc_code,enc_description,enc_reasoncode,enc_reasondescription
0,aaf736cb-3879-4b6d-9c48-cfcc7e81633d,2010-06-10,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,170258001,Outpatient Encounter,NaN,NaN
1,c18753b2-53ea-4e8f-bf69-2352a1dd980d,2010-07-28,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,170258001,Outpatient Encounter,NaN,NaN
2,3451074c-0452-4072-b814-ed7b9e05e811,2010-10-29,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,170258001,Outpatient Encounter,NaN,NaN
3,c59508aa-0063-482c-8645-b4b76a6e1a4e,2010-11-01,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,185345009,Encounter for symptom,195662009.0,Acute viral pharyngitis (disorder)
4,ff14c194-81e6-47db-856d-25d28c4ae028,2011-01-29,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,170258001,Outpatient Encounter,NaN,NaN


In [119]:
pat = pd.read_csv('./csv/patients.csv', 
            usecols=['BIRTHDATE', 'DEATHDATE', 'ID', 'MARITAL', 'GENDER']) \
           [['BIRTHDATE', 'DEATHDATE', 'MARITAL', 'GENDER', 'ID']]
pat.columns = [col.lower() for col in pat.columns]
pat.rename({'id':'patient'}, axis='columns', inplace=True)

pat.head()

,birthdate,deathdate,marital,gender,patient
0,2010-01-24,NaN,NaN,M,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0
1,2010-01-19,NaN,NaN,F,a875a900-eb20-4bb5-9183-7151cc840985
2,1953-11-21,1971-01-02,NaN,F,5269d81f-b7d6-49af-811d-e3bd8cc9d4b9
3,2008-10-14,NaN,NaN,M,7377a808-6a0f-45e2-868c-fce09c3622dd
4,1965-08-05,2004-01-15,M,M,cb6165f9-2d95-4abf-ab5a-a633cadd046b


In [120]:
f = pat['gender'] != 'M'
m = pat['gender'] != 'F'

In [121]:
corrupted = list(pat[f & m].index)

In [122]:
pat.drop(corrupted, axis=0, inplace=True)

In [123]:
w = pat['marital'] != 'S'
u = pat['marital'] != 'M'
v = pat['marital'].notnull()

In [124]:
corrupted1 = list(pat[w & u & v].index)

In [125]:
pat.drop(corrupted1, axis=0, inplace=True)

In [126]:
corrupted2 = list(pat[(pat['birthdate'].map(lambda x: len(x) != 10))].index)

In [127]:
pat.drop(corrupted2, axis=0, inplace=True)

In [128]:
l = pat['deathdate'].map(lambda x: len(str(x)) !=10)
n = pat['deathdate'].notnull()

In [129]:
corrupted3 = list(pat[l & n].index)

In [130]:
pat.drop(corrupted3, axis=0, inplace=True)

In [131]:
pat.dtypes

birthdate    object
deathdate    object
marital      object
gender       object
patient      object
dtype: object

Format the conditions dataframe.

In [132]:
con_map = dict([(col, 'con_'+col) for col in con.columns if col not in ['encounter', 'patient']])
con_map.update({'description':'condition'})
con.rename(mapper=con_map, axis='columns', inplace=True)

con.head()

,con_start,con_stop,patient,encounter,con_code,condition
0,2010-11-01,2010-11-12,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,c59508aa-0063-482c-8645-b4b76a6e1a4e,195662009,Acute viral pharyngitis (disorder)
1,2012-03-12,2012-03-26,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,149d165b-aed8-48d4-88d3-d9272ceb6664,43878008,Streptococcal sore throat (disorder)
2,2013-09-04,2013-09-15,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,711cc1a1-8529-4e87-8233-0f61396f8c87,444814009,Viral sinusitis (disorder)
3,2011-01-28,2011-02-16,a875a900-eb20-4bb5-9183-7151cc840985,3983a37e-56bc-4c59-897a-1a6c7a810561,10509002,Acute bronchitis (disorder)
4,2013-02-26,2013-04-18,a875a900-eb20-4bb5-9183-7151cc840985,a87c1299-a7ce-4310-9f2f-d88c15a159dc,65363002,Otitis media


Procedures dataframe.

In [133]:
pro = pd.read_csv('./csv/procedures.csv')
pro.columns = [col.lower() for col in pro.columns]
pro_map = dict([(col, 'pro_'+col) for col in pro.columns if col not in ['encounter', 'patient']])
pro_map.update({'description':'procedure'})
pro.rename(mapper=pro_map, axis='columns', inplace=True)

pro.head()

,pro_date,patient,encounter,pro_code,procedure,pro_reasoncode,pro_reasondescription
0,2010-06-10,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,aaf736cb-3879-4b6d-9c48-cfcc7e81633d,428191000124101,Documentation of current medications,NaN,NaN
1,2011-04-11,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,1f234324-33ca-400f-a6a5-6238640e2c62,428191000124101,Documentation of current medications,NaN,NaN
2,2011-10-08,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,112240a5-d8d5-42b2-9d31-c8b904ca9103,428191000124101,Documentation of current medications,NaN,NaN
3,2012-09-15,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,62be805c-4923-4fe6-b873-f3460851473b,428191000124101,Documentation of current medications,NaN,NaN
4,2014-02-12,2fa4d28e-b3a7-4b8b-ad3b-ef9cfc77fab0,5805dac8-b6ac-431a-8b30-6fffb855e8ce,428191000124101,Documentation of current medications,NaN,NaN


Merge Patients with Readings.

In [134]:
patients = pd.merge(pat, readings, how='inner', on=['patient'])

In [135]:
patients.head()

,birthdate,deathdate,marital,gender,patient,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Chloride,Creatinine,Diastolic Blood Pressure,Estimated Glomerular Filtration Rate,FEV1/​FVC,Glucose,High Density Lipoprotein Cholesterol,Low Density Lipoprotein Cholesterol,Microalbumin Creatine Ratio,Oral temperature,Potassium,Sodium,Systolic Blood Pressure,Total Cholesterol,Triglycerides,Urea Nitrogen,encounter,obs_date
0,1963-03-07,NaN,M,M,33025806-4630-46da-a5ff-ee0d9e20465a,181.93,30.33,100.40,9.05,20.0,108.0,1.0,74.0,NaN,NaN,70.0,NaN,NaN,NaN,NaN,4.32,144.0,113.0,NaN,NaN,13.0,09b351bb-3a6e-4634-9a82-200bbef6f1b9,2014-12-30
1,1963-03-07,NaN,M,M,33025806-4630-46da-a5ff-ee0d9e20465a,181.93,29.50,97.66,10.08,25.0,108.0,1.0,85.0,NaN,NaN,98.0,68.0,104.0,NaN,NaN,4.83,143.0,129.0,200.0,136.0,14.0,0afe4a0e-549c-413f-91c9-0fa6abdcf412,2010-07-21
2,1963-03-07,NaN,M,M,33025806-4630-46da-a5ff-ee0d9e20465a,181.93,30.33,100.40,9.68,20.0,102.0,1.0,76.0,NaN,NaN,88.0,NaN,NaN,NaN,NaN,4.69,140.0,110.0,NaN,NaN,12.0,1e9cfd92-e46a-4f43-ac60-89ab9e1eaf89,2015-10-21
3,1963-03-07,NaN,M,M,33025806-4630-46da-a5ff-ee0d9e20465a,181.93,30.33,100.40,9.17,27.0,105.0,1.0,73.0,NaN,NaN,76.0,NaN,NaN,NaN,NaN,4.82,143.0,102.0,NaN,NaN,11.0,3814175f-baac-43f1-98d3-f03b610fa494,2012-07-20
4,1963-03-07,NaN,M,M,33025806-4630-46da-a5ff-ee0d9e20465a,181.93,30.33,100.40,8.98,23.0,101.0,1.0,82.0,NaN,NaN,97.0,69.0,86.0,NaN,NaN,4.78,137.0,123.0,182.0,132.0,20.0,89692f83-203f-4b67-9f3a-176cafbf6f93,2014-01-27


In [136]:
f = patients['gender'] != 'M'
m = patients['gender'] != 'F'

In [137]:
patients[f & m]

,birthdate,deathdate,marital,gender,patient,Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Chloride,Creatinine,Diastolic Blood Pressure,Estimated Glomerular Filtration Rate,FEV1/​FVC,Glucose,High Density Lipoprotein Cholesterol,Low Density Lipoprotein Cholesterol,Microalbumin Creatine Ratio,Oral temperature,Potassium,Sodium,Systolic Blood Pressure,Total Cholesterol,Triglycerides,Urea Nitrogen,encounter,obs_date


Merge all data.

In [138]:
temp = enc.merge(pro, how='inner', on=['patient', 'encounter']) \
    .merge(con, how='inner', on=['patient', 'encounter'])

In [139]:
temp.head()

,encounter,enc_date,patient,enc_code,enc_description,enc_reasoncode,enc_reasondescription,pro_date,pro_code,procedure,pro_reasoncode,pro_reasondescription,con_start,con_stop,con_code,condition
0,220cdab0-1944-4700-8af3-002bfd66b742,2015-02-03,7a076378-ccdf-447d-aab9-20d3f7ad27fb,185345009,Encounter for symptom,10509002.0,Acute bronchitis (disorder),2015-02-03,23426006,Measurement of respiratory function (procedure),10509002.0,Acute bronchitis (disorder),2015-02-01,2015-02-11,10509002,Acute bronchitis (disorder)
1,0afe4a0e-549c-413f-91c9-0fa6abdcf412,2010-07-21,33025806-4630-46da-a5ff-ee0d9e20465a,185349003,Outpatient Encounter,NaN,NaN,2010-07-21,428191000124101,Documentation of current medications,NaN,NaN,1992-10-28,NaN,15777000,Prediabetes
2,63ce9acf-4f70-46ea-b4eb-6f3f42c685ce,2010-09-09,6466a363-ceb7-4c13-88e9-6fc62aac14de,185349003,Outpatient Encounter,NaN,NaN,2010-09-09,428191000124101,Documentation of current medications,NaN,NaN,2009-08-28,NaN,40055000,Chronic sinusitis (disorder)
3,91d30663-fdb7-40f2-9018-ab1de24d1555,2015-02-28,6466a363-ceb7-4c13-88e9-6fc62aac14de,185349003,Outpatient Encounter,NaN,NaN,2015-02-28,428191000124101,Documentation of current medications,NaN,NaN,2015-02-28,NaN,64859006,Osteoporosis (disorder)
4,91d30663-fdb7-40f2-9018-ab1de24d1555,2015-02-28,6466a363-ceb7-4c13-88e9-6fc62aac14de,185349003,Outpatient Encounter,NaN,NaN,2015-02-28,312681000,Bone density scan (procedure),NaN,NaN,2015-02-28,NaN,64859006,Osteoporosis (disorder)


In [140]:
df = pd.merge(patients, temp, on=['encounter', 'patient'], how='inner')

In [142]:
df.to_csv('./csv/diabetic_dirty.csv', index=False)